# <center> <b> <span style="color: hotpink;">  ShadeSense </center> #

<center>


![](https://github.com/ginaguerin/ShadeSense_Lipstick_Shade_Identifier_App/blob/master/logos/logo3.2.jpeg?raw=true)


## <u> Concept: </u> ##

- ShadeSense is an innovative application designed to instantly identify the specific shade of lipstick someone is wearing in real-time. The ultimate goal of this application is to create an extensive library capable of recognizing a wide range of lipstick shades and brands. Importantly, ShadeSense aims to be inclusive, ensuring accurate detection across all lipstick colors, irrespective of gender or skin tone.

## <u> Scope: </u> ##

- In its initial phase, ShadeSense will begin with a curated selection from the renowned makeup brand, MAC Cosmetics. This curated collection consists of five distinct lipstick shades, serving as a starting point for the app's development. As TwinTone evolves, it will expand its library to encompass a diverse array of lipstick shades from various brands, further enhancing its capabilities.

## Dataset Overview: ##

- **Images:** The dataset comprises original resized images in JPG format. These images are the input data for training the lipstick shade identification model.

- **Annotations:** Each image is associated with an annotated XML file. These XML files likely contain information about the location and class labels of the lipstick shades within the corresponding image.

- **Classes:** There are 7 distinct classes corresponding to different lipstick shades. The goal is to train the model to recognize and classify these lipstick shades automatically.

- **Supervised Learning:** The dataset is suitable for supervised learning, where the model learns from the paired examples of images and their corresponding annotations to generalize and make predictions on new, unseen data.

- **Training Objective:** The objective of the model is to analyze the images and identify the correct lipstick shade class based on the provided annotations.

- **Neural Network Architecture:** The CNN architecture outlined in the baseline model is designed to process the image data and make predictions for the 7 lipstick shade classes.


## <u> <span style="color: red;"> Limitations: </span> </u> ##


1. **Limited Dataset Size:**
   - With only 220 images for the six lipstick shades, including images of no lipstick, the dataset might be relatively small for training a highly accurate and robust neural network.
2. **Model Generalization:**
   - The model's ability to generalize to different lipstick shades, brands, and skin tones might be limited initially. Training on a diverse dataset can help improve generalization.

3. **Brand and Shade Specificity:**
   - The initial focus on MAC Cosmetics and six specific shades may limit the app's applicability to users with different preferences or those using other brands. Expanding the dataset to include various brands and shades can address this limitation over time.

4. **Real-Time Processing Constraints:**
   - Real-time processing on live camera feeds can be computationally intensive, leading to potential performance limitations, especially on devices with lower processing power. Optimizing the model and deploying it on platforms that support efficient real-time processing.

5. **Lighting and Environmental Conditions:**
   - The accuracy of the lipstick detection may be influenced by lighting conditions and the environment. Variations in lighting may impact color perception, potentially affecting the model's performance.

6. **User Privacy Concerns:**
   - The app involves capturing and processing images in real-time, raising privacy concerns. Making sure to clearly communicate to users how their data will be handled, stored, and if any images are stored temporarily for processing.

7. **Device Compatibility:**
   - The app's real-time detection capabilities may be influenced by the camera quality and specifications of the user's device. Ensure compatibility across a range of devices and optimize the app's performance accordingly.

8. **Legal and Ethical Considerations:**
   - Ensure compliance with legal and ethical standards, especially when dealing with image data. Being aware of privacy laws, data protection regulations, and obtain consent when necessary.

9. **Feedback and Iterative Development:**
   - Given the initial focus on a smaller dataset and makeup brand, user feedback will be crucial for identifying limitations and areas for improvement. Planning for iterative development to enhance the app based on user experiences and preferences.

In [1]:
#import necessary libraries
import os
import pandas as pd
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers, optimizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.applications import VGG16

2024-01-02 11:27:46.401887: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Image Data Preprocessing ###


1. **File Discovery**
   

2. **Annotation Pairing**


3. **Class Extraction**
   

4. **Image Loading and Conversion**
   

5. **Final Preprocessing**
   

In [2]:
folder_path = "images"

# Get a list of all files in the folder
all_files = os.listdir(folder_path)

# Filter only image files
image_files = [f for f in all_files if f.lower().endswith(".jpg")]

# Create a DataFrame with columns "filename" and "annotation"
df = pd.DataFrame({'filename': image_files})

# Corresponding XML file with the same name
df['annotation'] = df['filename'].apply(lambda x: os.path.splitext(x)[0] + ".xml")

# Extract class name from XML
def extract_class_name(xml_path):
    try:
        tree = ET.parse(os.path.join(folder_path, xml_path))
        root = tree.getroot()
        
        
        object_name_element = root.find('.//object/name')
        class_name = object_name_element.text if object_name_element is not None else None
        
        return class_name
    except Exception as e:
        print(f"Error extracting class name from {xml_path}: {str(e)}")
        return None

# Create a new column "class_name"
df['class_name'] = df['annotation'].apply(extract_class_name)

# Load and preprocess images
def load_and_preprocess_image(image_path, target_size=(512, 512)):
    try:
        # Load the image and resize it
        image = Image.open(os.path.join(folder_path, image_path))
        image = image.resize(target_size)
        
        # Convert the image to a NumPy array
        image_data = np.array(image)
        
        # Convert the NumPy array to a TensorFlow tensor
        image_tensor = tf.convert_to_tensor(image_data, dtype=tf.float32)
        
        return image_tensor
    except Exception as e:
        print(f"Error loading image {image_path}: {str(e)}")
        return None

# Create a new column "image_data"
df['image_data'] = df['filename'].apply(load_and_preprocess_image)


# Preprocess image data
def preprocess_image(image_data, target_size=(512, 512)):
    try:
        # Convert the NumPy array to a TensorFlow tensor
        image = tf.convert_to_tensor(image_data, dtype=tf.float32)
        
        # Resize and preprocess the image
        image = tf.image.resize(image, target_size)
        image = tf.keras.applications.vgg16.preprocess_input(image)
        
        return image
    except Exception as e:
        print(f"Error preprocessing image data: {str(e)}")
        return None

# Preprocessing to image data
df['image'] = df['image_data'].apply(preprocess_image)



### Data Splitting ###


1. **Dataset Splitting:**
   - The `train_test_split` function is utilized to partition the dataset into training and test sets.

2. **Set Size Information:**
   - The code prints the size of both the training and test sets, providing insights into the distribution of data between the two subsets.


In [3]:
# Split the data into training and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Print the number of items in each set
print("Training set size:", len(train_df))
print("Test set size:", len(test_df))

Training set size: 176
Test set size: 44


### Data Preprocessing ###

**Handling Missing Labels**
   

2. **Label Encoding**
   

3. **Image Normalization**
   

4. **Remaining Items Information**
   

5. **Unique Labels Verification**
   

In [4]:
# Handle missing labels by dropping rows with missing labels
train_df = train_df.dropna(subset=['class_name'])
test_df = test_df.dropna(subset=['class_name'])

# Reapply the encoding
label_encoder = LabelEncoder()
encoded_labels_train = label_encoder.fit_transform(train_df['class_name'])
encoded_labels_test = label_encoder.transform(test_df['class_name'])

# Normalize images in the training set
train_images = np.stack(train_df['image'].to_numpy())
train_images_normalized = train_images / 255.0 

# Normalize images in the test set using the same parameters as the training set
test_images = np.stack(test_df['image'].to_numpy())
test_images_normalized = test_images / 255.0 


# Print the number of items remaining after dropping rows
print("Number of items remaining in the training set:", len(train_df))
print("Number of items remaining in the test set:", len(test_df))

# Print unique labels for verification
print("Unique labels in training set:", np.unique(encoded_labels_train))
print("Unique labels in the test set:", np.unique(encoded_labels_test))


Number of items remaining in the training set: 175
Number of items remaining in the test set: 44
Unique labels in training set: [0 1 2 3 4 5 6]
Unique labels in the test set: [0 1 2 3 4 5 6]


# Baseline Model

The baseline model is meticulously crafted for the lipstick shade identification task using a dataset comprising original resized JPG images and their corresponding annotated XML files. Here are the key aspects of the baseline model, with a focus on monitoring the F-1 score due to our small dataset, imbalanced classes, and concerns for misclassifying categories:

- **Model Architecture:** A Convolutional Neural Network (CNN) is tailored for image processing, featuring three convolutional layers (32, 64, and 128 filters), subsequent max-pooling layers, a flatten layer, and two dense layers. The model is specifically designed to process images with dimensions (512, 512, 3).

- **Dataset Structure:** Our dataset consists of paired examples, linking original resized images to annotated XML files. Each image is enriched with details about the location and class labels of various lipstick shades.

- **Supervised Learning:** The model adopts a supervised learning approach, learning to make predictions based on annotated images, with the objective of automatically categorizing lipstick shades into 7 distinct classes.

- **Training Objective:** The primary aim is to train the model for accurate recognition and classification of lipstick shades. Ground truth information from annotated XML files serves as the foundation for model training.

- **Model Compilation:** The model is compiled using the Adam optimizer, sparse categorical crossentropy loss function, and includes F-1 score as an additional evaluation metric. This modification aligns with our focus on addressing the challenges posed by our small dataset and imbalanced classes.

- **Training Process:** The model undergoes training for 10 epochs using the paired dataset, optimizing parameters to minimize crossentropy loss and maximize the F-1 score. The validation dataset plays a crucial role in evaluating the model's generalization to unseen data.

This baseline model acts as a pivotal step in constructing a robust lipstick shade identification system. It lays the groundwork for subsequent enhancements and tuning efforts, ensuring our model evolves and performs optimally on our specific dataset.

In [7]:
# Define the CNN model with normalized input
num_classes = len(np.unique(encoded_labels_train))

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(512, 512, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model with normalized training set
history = model.fit(
    train_images_normalized,
    encoded_labels_train,
    epochs=10,
    validation_data=(test_images_normalized, encoded_labels_test)
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_images_normalized, encoded_labels_test, verbose=0)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

# Predict the labels for the test set
y_pred = model.predict(test_images_normalized)
y_pred_classes = np.argmax(y_pred, axis=1)

# Convert encoded labels back to original labels
original_labels_test = label_encoder.inverse_transform(encoded_labels_test)
predicted_labels_test = label_encoder.inverse_transform(y_pred_classes)

# Print classification report
print("\nClassification Report:")
print(classification_report(original_labels_test, predicted_labels_test))


Epoch 1/10
6/6 [==============================] - 119s 18s/step - loss: 5.7034 - accuracy: 0.1600 - val_loss: 1.8986 - val_accuracy: 0.3864
Epoch 2/10
6/6 [==============================] - 82s 13s/step - loss: 1.8783 - accuracy: 0.3371 - val_loss: 1.8532 - val_accuracy: 0.3409
Epoch 3/10
6/6 [==============================] - 85s 14s/step - loss: 1.5384 - accuracy: 0.5143 - val_loss: 1.5259 - val_accuracy: 0.3864
Epoch 4/10
6/6 [==============================] - 82s 13s/step - loss: 1.0088 - accuracy: 0.6800 - val_loss: 1.3059 - val_accuracy: 0.5455
Epoch 5/10
6/6 [==============================] - 86s 14s/step - loss: 0.4990 - accuracy: 0.8343 - val_loss: 1.1816 - val_accuracy: 0.7273
Epoch 6/10
6/6 [==============================] - 74s 12s/step - loss: 0.2387 - accuracy: 0.9543 - val_loss: 0.7935 - val_accuracy: 0.7727
Epoch 7/10
6/6 [==============================] - 86s 14s/step - loss: 0.0848 - accuracy: 0.9829 - val_loss: 0.7612 - val_accuracy: 0.7727
Epoch 8/10
6/6 [==========

### Observations from Baseline Model: ###

Upon reviewing the baseline model, certain observations and key metrics were identified, highlighting aspects of overfitting and performance during training:

- **Epoch 1/10:**
  - Training Loss: 5.7034, Training Accuracy: 16%
  - Validation Loss: 1.8986, Validation Accuracy: 39%

- **Epoch 10/10:**
  - Training Loss: 0.0174, Training Accuracy: 99%
  - Validation Loss: 0.8105, Validation Accuracy: 75%

### Anticipated Improvements with Enhanced Model: ###
For an enhanced model, specific strategies were proposed to address overfitting and enhance overall performance:

1. **Overfitting Mitigation:**
   - The introduction of L2 regularization and dropout layers is anticipated to alleviate overfitting concerns.

2. **Balanced Accuracy:**
   - Expectations are set for improved balance in accuracy across both training and validation sets.


3. **Challenges and Opportunities:**
   - The model performs well on certain classes but encounters challenges, particularly with "Whirl," where precision and recall are lower.The imbalanced distribution of classes, especially the smaller classes, can influence the model's ability to generalize.




## Anticipating Enhanced Model Performance: ##

- **Model Type:** Convolutional Neural Network (CNN)
- **Enhancements:**
  - L2 Regularization: Integrated with a weight decay of 0.01
  - Dropout Layers: Strategically placed with dropout rates of 0.25 and 0.5
- **Objective:** Fortify against overfitting and enhance adaptability

### Evaluation Metrics Outlook: ###
Anticipated evaluation metrics for the enhanced model:

- **Test Loss: 1.2101, Test Accuracy: 77%**

In [16]:
# Create and fit label encoder
label_encoder = LabelEncoder()
encoded_labels_train = label_encoder.fit_transform(train_df['class_name'])
encoded_labels_test = label_encoder.transform(test_df['class_name'])

# Define the CNN model with regularization and dropout
num_classes = len(np.unique(encoded_labels_train))

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(512, 512, 3), kernel_regularizer=regularizers.l2(0.01)),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25), 

    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.5),  
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model with the Adam optimizer
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    np.stack(train_df['image'].to_numpy()),
    encoded_labels_train,
    epochs=10,
    validation_data=(np.stack(test_df['image'].to_numpy()), encoded_labels_test)
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(np.stack(test_df['image'].to_numpy()), encoded_labels_test, verbose=0)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

# Predict the labels for the test set
y_pred = model.predict(np.stack(test_df['image'].to_numpy()))
y_pred_classes = np.argmax(y_pred, axis=1)

# Convert encoded labels back to original labels
original_labels_test = label_encoder.inverse_transform(encoded_labels_test)
predicted_labels_test = label_encoder.inverse_transform(y_pred_classes)

# Print classification report
print("\nClassification Report:")
print(classification_report(original_labels_test, predicted_labels_test, zero_division=1))


Epoch 1/10
6/6 [==============================] - 105s 14s/step - loss: 3419.6638 - accuracy: 0.1429 - val_loss: 32.6378 - val_accuracy: 0.1591
Epoch 2/10
6/6 [==============================] - 99s 16s/step - loss: 74.9371 - accuracy: 0.1486 - val_loss: 15.0598 - val_accuracy: 0.2045
Epoch 3/10
6/6 [==============================] - 83s 13s/step - loss: 16.4527 - accuracy: 0.2171 - val_loss: 18.2549 - val_accuracy: 0.3409
Epoch 4/10
6/6 [==============================] - 98s 15s/step - loss: 18.9322 - accuracy: 0.2914 - val_loss: 20.1825 - val_accuracy: 0.4091
Epoch 5/10
6/6 [==============================] - 82s 14s/step - loss: 20.4786 - accuracy: 0.3257 - val_loss: 21.2733 - val_accuracy: 0.5000
Epoch 6/10
6/6 [==============================] - 78s 12s/step - loss: 21.0242 - accuracy: 0.4971 - val_loss: 21.6287 - val_accuracy: 0.5000
Epoch 7/10
6/6 [==============================] - 84s 14s/step - loss: 21.0958 - accuracy: 0.6229 - val_loss: 21.9288 - val_accuracy: 0.4773
Epoch 8/10

## Summary of Enhanced Model Performance ##

### Model Architecture Enhancements: ###
The enhanced model introduced several architectural modifications, including dropout layers and L2 regularization, to address overfitting concerns. However, the model's performance in terms of accuracy and key metrics did not show significant improvement.

### Training and Evaluation Results: ###
The training and validation results over 10 epochs for the enhanced model are outlined below:

- **Epoch 1/10:**
  - Training Loss: 3419.66, Training Accuracy: 14.29%
  - Validation Loss: 32.64, Validation Accuracy: 15.91%

- **Epoch 10/10:**
  - Training Loss: 20.5985, Training Accuracy: 90.29%
  - Validation Loss: 21.3792, Validation Accuracy: 68.18%

### Performance Metrics: ###
The classification report provides insights into the model's capability to classify lipstick shade categories. Key observations include:

- **Classification Report:**
  - "Crème D'Nude" and "None" show high precision, recall, and F-1 scores, indicating strong performance. 
  - "Honey Love" and "Whirl" face challenges, with lower F-1 scores, suggesting potential areas for improvement.
  - Overall accuracy is 68.18%, indicating some improvements in correctly classifying lipstick shades.

### Observations and Next Steps: ###
1. Despite architectural enhancements, the model's performance remains suboptimal, with limited improvements in accuracy.
2. Further exploration of hyperparameters, model complexity, and potential consideration of alternative architectures (e.g., deeper networks or transfer learning) could be pivotal in addressing the observed limitations.
3. Continue model refinement and fine-tuning are recommended to achieve more desirable classification outcomes.

## Model Tuning for Improved Performance ##

- **Model Type:** Convolutional Neural Network (CNN) based on VGG16
- **Enhancements:**
  - L2 Regularization: Applied with a weight decay of 0.0001
  - Dropout Layers: Strategically placed with a dropout rate of 0.1
  - Learning Rate: Adjusted to 0.0001
- **Epoch Cycle:** Set to 10 epochs

Our strategy for model enhancement involves leveraging L2 regularization, dropout layers, and a custom learning rate. Additionally, data augmentation is applied to increase the model's robustness and generalization capability.


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
 
# Create and fit label encoder
label_encoder = LabelEncoder()
encoded_labels_train = label_encoder.fit_transform(train_df['class_name'])
encoded_labels_test = label_encoder.transform(test_df['class_name'])

# Define the CNN model with regularization and dropout
num_classes = len(np.unique(encoded_labels_train))

# Load the VGG16 model without the top (fully connected) layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(512, 512, 3))

# Freeze the convolutional layers of VGG16
for layer in base_model.layers:
    layer.trainable = False

# Create a new model and add the VGG16 base model with dropout
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(186, activation='relu', kernel_regularizer=regularizers.l2(0.0001)),
    layers.Dropout(0.1),
    layers.Dense(num_classes, activation='softmax')
])

# Unfreeze the last 3 layers of the base model
for layer in base_model.layers[-3:]:
    layer.trainable = True

# Compile the model with the custom optimizer (Adam with a learning rate of 0.0001)
custom_optimizer = optimizers.Adam(learning_rate=0.0001)
from tensorflow.keras import metrics

# Assuming you have a custom optimizer named custom_optimizer
model.compile(optimizer=custom_optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy', metrics.Precision(), metrics.Recall(), metrics.F1Score()])


# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,  
    width_shift_range=0.1,  
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1, 
    horizontal_flip=True,
    fill_mode='nearest'
)

# Train the model with data augmentation
history = model.fit(
    datagen.flow(np.stack(train_df['image'].to_numpy()), encoded_labels_train, batch_size=28),
    epochs=10,
    validation_data=(np.stack(test_df['image'].to_numpy()), encoded_labels_test)
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(np.stack(test_df['image'].to_numpy()), encoded_labels_test, verbose=0)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

# Predict the labels for the test set
y_pred = model.predict(np.stack(test_df['image'].to_numpy()))
y_pred_classes = np.argmax(y_pred, axis=1)

# Convert encoded labels back to original labels
original_labels_test = label_encoder.inverse_transform(encoded_labels_test)
predicted_labels_test = label_encoder.inverse_transform(y_pred_classes)

# Print classification report
print("\nClassification Report:")
print(classification_report(original_labels_test, predicted_labels_test, zero_division=1))

# Save the label encoder classes
np.save('models/label_encoder_classes2.npy', label_encoder.classes_)

Epoch 1/10
7/7 [==============================] - 335s 48s/step - loss: 9.1102 - accuracy: 0.1886 - val_loss: 3.4273 - val_accuracy: 0.3864
Epoch 2/10
7/7 [==============================] - 410s 59s/step - loss: 2.2114 - accuracy: 0.2914 - val_loss: 1.8295 - val_accuracy: 0.2727
Epoch 3/10
7/7 [==============================] - 401s 58s/step - loss: 1.6334 - accuracy: 0.3829 - val_loss: 1.6145 - val_accuracy: 0.3636
Epoch 4/10
7/7 [==============================] - 388s 56s/step - loss: 1.5214 - accuracy: 0.4457 - val_loss: 1.6219 - val_accuracy: 0.5000
Epoch 5/10
7/7 [==============================] - 372s 52s/step - loss: 1.2882 - accuracy: 0.5714 - val_loss: 1.4234 - val_accuracy: 0.5227
Epoch 6/10
7/7 [==============================] - 372s 53s/step - loss: 1.0194 - accuracy: 0.6571 - val_loss: 1.1056 - val_accuracy: 0.6364
Epoch 7/10
7/7 [==============================] - 376s 61s/step - loss: 0.9671 - accuracy: 0.6914 - val_loss: 0.8625 - val_accuracy: 0.7273
Epoch 8/10
7/7 [====

## Summary of Final Model with Lowered Dropout Rate ##

### Model Architecture and Training Dynamics: ###
The model, incorporating VGG16 with a lowered dropout rate of 0.38, demonstrates improved stability and generalization during an 8-epoch training regimen:

## Interpretation of Model Results ##

- **Epoch 1/8:**
  - Training Loss: 9.11, Training Accuracy: 18.86%
  - Validation Loss: 3.43, Validation Accuracy: 38.64%

- **Epoch 8/8:**
  - Training Loss: 0.43, Training Accuracy: 81.82%
  - Validation Loss: 0.43, Validation Accuracy: 81.82%

### Test Set Performance: ###
The model demonstrates promising results on the test set, emphasizing robustness and generalization.

- **Test Loss:** 0.43, **Test Accuracy:** 81.82%

### Classification Report: ###
The classification report provides a detailed breakdown of the model's performance across makeup product categories.

- **Key Observations:**
  - Precision, recall, and F1-score metrics exhibit improvements, especially for classes like "Honey Love," "Ruby Woo," and "Whirl."
  - The overall accuracy is 81.82%, highlighting the model's capability to correctly classify makeup products.

### Conclusion: ###
The new model results reflect substantial enhancements, emphasizing improved accuracy and class-specific metrics. This suggests that the model has successfully learned intricate patterns within the data, showcasing its potential for accurate lipstick shade.

In [6]:
# Save the model in the native Keras format
model.save('models/shadesense_final_model4')

INFO:tensorflow:Assets written to: models/shadesense_final_model4/assets


INFO:tensorflow:Assets written to: models/shadesense_final_model4/assets


## Conclusion and Future Steps ##

The culmination of our model tuning efforts has resulted in a highly promising Convolutional Neural Network (CNN) based on VGG16. Leveraging strategies such as L2 regularization, dropout layers, and a custom learning rate, our model exhibits exceptional performance with a test accuracy of 81.82%. The classification report further emphasizes improved precision, recall, and F1-score metrics across various lipstick shades.

Moving forward, we plan to integrate this finalized model into our ShadeSense Lipstick Shade Identifier App. Real-world usage will provide valuable insights into the model's practical performance, allowing us to analyze its strengths and identify areas for further refinement. Additionally, we remain committed to continuous improvement by fine-tuning the model, expanding our dataset, and exploring opportunities to enhance its capabilities.

Excitement surrounds the prospect of observing the model's progression in real-world scenarios and its adaptability to a diverse range of user-generated inputs. This iterative approach ensures that our app evolves with user interactions, delivering accurate and reliable results for lipstick shade identification. We look forward to the app's continued development, with a keen eye on user feedback and data augmentation to propel its performance to new heights.